In [1]:
from datetime import datetime
import requests
import sys
import numpy as np
import pandas as pd

sys.path.append('..')
import moonlander

from bokeh.io import output_notebook, show, push_notebook
output_notebook()

Loading BokehJS ...

In [2]:
import moonlander as mld
from moonlander.data.providers import Binance

api_key = 'JUY2Z2R1Lwpgta6toGtB8ofCxpiSBEiqqHJM8rp4GCWzIy2kXFbERuvxB2dDD1Cj'
secret_key = 'hDv51FaiLDHEA7Al2zhuG9YlXoKmgntjvgZDofFKNis1hY27x4WXHoL1RIsf1e8F'

binance = Binance(api_key, secret_key)

In [4]:
btc = moonlander.core.Asset('DOGE', 'USDT', provider = binance, interval = '15-min', start_date = '2021-05-01 00:00:00')

All caught up..!


In [5]:
# btc.plot_candles()

In [15]:
from moonlander.strategy import TradingStrategy
from moonlander.indicators import Indicator, SimpleMovingAverage, ExponentialMovingAverage, BollingerBands

class MACDCrossover(TradingStrategy):
    def setup(self):
        self.macd = moonlander.indicators.MACD(fastperiod = 6, slowperiod = 18, signalperiod = 5, plot_separately = True)
        
        self.indicators = [self.macd]
    
    def logic(self, account, lookback):
        try:
            today = lookback.iloc[-1]
            
            macd_today, signal_today, _ = self.macd.lookback[-1]
            macd_yest, signal_yest, _ = self.macd.lookback[-2]
            
            # Buying
            buy_signal = (macd_today > signal_today) and (macd_yest < signal_yest)
            if buy_signal:
                entry_price   = today.close
                entry_capital = account.buying_power
                account.buy(entry_capital=entry_capital, entry_price=entry_price)
                
            # Selling
            sell_signal = (macd_today < signal_today) and (macd_yest > signal_yest)
            
            if sell_signal:
                percent = 1
                exit_price = today.close 
                account.sell(percent=percent, current_price=exit_price)
                
        except Exception as e:
#             print('ERROR', e)
            pass

In [16]:
strat = MACDCrossover(btc)

In [17]:
strat.backtest(start_date = '2021-06-1 00:00:00', commision = 0.001)

Performing backtest from: 01 June, 2021 (00:00:00) to 17 June, 2021 (20:00:00)



-------------- Results ----------------

Relative Returns: 7.88%
Relative Profit: 78.77

Strategy     : 1.61%
Net Profit   : 16.14

Buy and Hold : -6.26%
Net Profit   : -62.63

Buys        : 102
Sells        : 101
--------------------
Total Trades : 203

---------------------------------------


In [18]:
coindcx_api_key = 'fcb1c800d0ee02c93ea14bf61517afa0194d5c695eec9792'
coindcx_api_secret = '3071e32a059b72505f7d261d95aacb1ba66ab8e4cf5e89214bf329b454196c4c'

coindcxaccount = moonlander.account.coindcx.CoinDCX(btc, coindcx_api_key, coindcx_api_secret)

In [19]:
strat.execute(trading_account = coindcxaccount, plot = False)

All caught up..!
